# 1. Settings
## 1) import required libraries

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.utils.data as data
import torchvision.datasets as dset
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import time
import matplotlib.pyplot as plt
import utils
import torchvision

## 2) Hyper parameters

In [3]:
batch_size = 16
learning_rate = 1e-4
epochs = 20

n_node = 1024
dropout_ratio = 0.5

img_size = 256

# 2. Data Loader
### training data

In [ ]:
img_dir = 'animal/train'
train_data = dset.ImageFolder()